<a href="https://colab.research.google.com/github/OwlSaver/GWU/blob/main/Praxis/Notebooks/Praxis_Test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goals


*   Convert a document to a knowledge graph
*   Merge knowlege graph into



Architecture


*   Input of Prompts or Documents
*   List item



To Do

*   Convert
*   List item



In [ ]:
!pip install python-docx
import docx
import os
import google.generativeai as genai
from google.colab import userdata
import json
import re
import textwrap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive to access the file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class GeminiInitializationError(Exception):
    pass

class GeminiQueryError(Exception):
    pass

class GeminiResponseBlockedError(Exception):
    pass

class ParagraphsTooLong(Exception):
    pass

In [ ]:
class Mind:
  INDENT = 4                    # The numer of characters to indent the json submitted to Gemini.
  MAX_VARIABLE_LENGTH = 2 ** 16 # Adjust as needed.  Consider the context window of your chosen model.
  PARAGRAPH_SET = 50            # The maximum number of paragraphs to process in a set.
  ITERATIONS = 3                # The number of times to process the data
  STM_MAX = 10000               # Once STM reaches this size it will be consolidated to LTM
  Q_KG = '''Create a separate knowledge graph for each named entity you find in the content.
            The knowledge graph must be in the following json schema format.
            {
              "$schema": "https://json-schema.org/draft/2020-12/schema",
              "$id": "https://example.com/product.schema.json",
              "title": "Frames in JSON.",
              "description": "An approach to use JSON for data in a Frame structure. See A Framework for Representing Knolege, Marvin Minsky, 1974.",
              "type": "array",
              "entities": {
                "type": "object",
                "properties": {
                  "entity": { "type": "string" },
                  "type": { "type": "string" },
                  "properties": {
                    "type": "array",
                    "items": {
                      "type": "object",
                      "properties": {
                        "propertyName": { "type": "string" },
                        "propertyValue": { "type": "string" }
                      },
                      "required": ["propertyName", "propertyValue"]
                    }
                  },
                  "relationships": {
                    "type": "array",
                    "items": {
                      "type": "object",
                      "properties": {
                        "relationshipType": { "type": "string" },
                        "relationshipTarget": { "type": "string" }
                      },
                      "required": ["relationshipType", "relationshipTarget"]
                    }
                  }
                },
                "required": ["name", "type", "properties", "relationships"]
              }
            }
      Do not include explanations of what you did. Use the schema to format your response.
      Do not include the schema in your rsponse. Just include the json in your response.'''
  Q_PC = "How many paragraphs are there in this content?"
  Q_MPC = "Review the content and assume that the estimates are correct. Then add them all togeather for a total count of paragraphs."
  Q_WC = "How many words are there in this content?"
  Q_MWC = "Review the content and assume that the estimates are correct. Then add them all togeather for a total count of words."
  Q_TW = "What are the ten most used words in this content and how often does each occur? Ignore function words for this count."
  Q_MTW = "Review the content and assume that the estimates are correct. Then merge them all togeather for a summarized list of words sorted by frequency."


  def __init__(self):
    self.ShortTermMemory = []
    self.LongTermMemory = []

  def text_to_stm(self, variable, question, api_key=None, model_name="gemini-1.5-flash", max_output_tokens=None, temperature=None,  additional_context=None):
    """
    Asks Gemini a question about the contents of a Python variable.

    Args:
        variable: The Python variable to analyze.
        question: The question to ask Gemini about the variable.
        api_key: (Optional) Your Gemini API key.  If not provided, it should be
                  available via genai.configure().
        model_name: (Optional) The Gemini model to use (e.g., "gemini-1.5-pro", "gemini-1.0-pro").  Defaults to "gemini-1.5-pro".
        max_output_tokens: (Optional) Maximum number of tokens in the response.
        temperature: (Optional) Controls the randomness of the response (0.0 - 1.0).  Lower values are more deterministic.
        additional_context: (Optional) A string containing additional information relevant to the variable or question.

    Returns:
        The Gemini API's response as a string, or an error message.

    Raises:
        GeminiInitializationError: If there's an issue initializing the Gemini model.
        GeminiQueryError: If there's an error querying the Gemini API.
        GeminiResponseBlockedError: If the Gemini response is blocked due to content policy.
    """

    if api_key:
        genai.configure(api_key=api_key)


    try:
        model = genai.GenerativeModel(model_name)
    except Exception as e:
      raise GeminiInitializationError(f"Error initializing Gemini model: {str(e)}. Ensure your API key is configured and you have access to the specified model.")

    # --- Improved Prompt Construction ---
    prompt_parts = []
    prompt_parts.append("You are a helpful assistant that can analyze Python variables that contain paragraphs of text and answer questions about them.")

    if additional_context:
        prompt_parts.append(f"Additional Context:\n{additional_context}\n")

    prompt_parts.append(f"Analyze the following Python variable and answer the question below:")

    # --- Improved Variable Handling ---

    # Use a safer representation:  json.dumps is generally better for complex objects.
    try:
        variable_str = json.dumps(variable, indent=self.INDENT, default=str)  # Use default=str to handle non-serializable objects
    except TypeError:
        variable_str = repr(variable) # Fallback to repr if json fails

    # Limit variable size to prevent long prompts
    variable_str = textwrap.shorten(variable_str, width=self.MAX_VARIABLE_LENGTH, placeholder="... (truncated)")

    prompt_parts.append(f"Variable (Python):\n`python\n{variable_str}\n`")

    prompt_parts.append(f"Question:\n{question}")

    # ---  Construct the full prompt  ---
    prompt = "\n\n".join(prompt_parts)

    # --- Generation Configuration ---
    generation_config = {}
    if max_output_tokens is not None:
        generation_config["max_output_tokens"] = max_output_tokens
    if temperature is not None:
        generation_config["temperature"] = temperature

        # Add more safety settings, these are very important
    safety_settings = [
      {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
    ]

    # ---  Generate Response  ---
    try:
        response = model.generate_content(prompt, generation_config=generation_config, safety_settings=safety_settings)
        #  Check for response.prompt_feedback to see if the prompt was blocked.
        if response.prompt_feedback:
          raise GeminiResponseBlockedError(f"Prompt was blocked: {response.prompt_feedback}")
        return response.text
    except Exception as e:
        # More specific error handling:  Catch common errors.
        if "Response was blocked" in str(e):  # Check for content filtering.
            raise GeminiResponseBlockedError(f"Error: Gemini response was blocked due to content policy. Error: {e}")
        raise GeminiQueryError(f"Error querying Gemini API: {str(e)}")


  def consolidate_stm_to_ltm(self):
    """
    Consolidate the Short Term Memory into the Long Term Memory.
    """
    print("Consolidating STM to LTM")
    new_JSON = ""
    for jsonString in self.ShortTermMemory:
      if new_JSON == "":
        new_JSON = jsonString
      else:
        new_JSON = self.merge_entities_lists(new_JSON, jsonString)
    if new_JSON != "":
      self.LongTermMemory.append(new_JSON)
    self.ShortTermMemory.clear()

  import json

  def merge_entities_lists(self, entities1_str, entities2_str):
      if not entities1_str:
          return entities2_str
      if not entities2_str:
          return entities1_str

      entities1 = json.loads(entities1_str)
      entities2 = json.loads(entities2_str)

      merged_entities = {}

      for entity in entities1 + entities2:
          key = (entity["entity"], entity["type"])
          if key not in merged_entities:
              merged_entities[key] = {
                  "entity": entity["entity"],
                  "type": entity["type"],
                  "properties": [],
                  "relationships": []
              }
          merged_entities[key]["properties"].extend(entity["properties"])
          merged_entities[key]["relationships"].extend(entity["relationships"])

      return json.dumps(list(merged_entities.values()), indent=2)


  def merge_jsons(self,json_str1, json_str2):
    """
    Merge two JSON strings with multiple entities based on the entity and type.

    Args:
        json_str1 (str): The first JSON string.
        json_str2 (str): The second JSON string.

    Returns:
        str: The merged JSON string.
    """
    print("Merging JSONs")
    #print(json_str1)
    #print(json_str2)
    # Parse the JSON strings
    data1 = json.loads(json_str1)
    data2 = json.loads(json_str2)

    # Create a dictionary to hold the merged entities
    merged_data = {}

    # Helper function to merge properties and relationships
    '''
    def merge_entities(entity1, entity2):
        merged_properties = {**entity1['properties'], **entity2['properties']}
        merged_relationships = {**entity1['relationships'], **entity2['relationships']}
        return {
            'entity': entity1['entity'],
            'type': entity1['type'],
            'properties': merged_properties,
            'relationships': merged_relationships
        }
    '''
    def merge_entities(entity1, entity2):
        merged_properties = {**entity1['properties'], **entity2['properties']}
        # Check if relationships are lists and convert to dictionaries
        relationships1 = entity1.get('relationships', {})  # Use get with default value to handle missing key
        relationships2 = entity2.get('relationships', {})  # Use get with default value to handle missing key
        # relationships1 = entity1['relationships']
        # relationships2 = entity2['relationships']
        if isinstance(relationships1, list):
            relationships1 = {i: v for i, v in enumerate(relationships1)}  # Assign numerical keys to list elements
        if isinstance(relationships2, list):
            relationships2 = {i: v for i, v in enumerate(relationships2)}  # Assign numerical keys to list elements

        merged_relationships = {**relationships1, **relationships2}
        return {
            'entity': entity1['entity'],
            'type': entity1['type'],
            'properties': merged_properties,
            'relationships': merged_relationships
        }
    # Process entities in the first JSON
    for item in data1:
        key = (item['entity'], item['type'])
        merged_data[key] = item

    # Process entities in the second JSON
    for item in data2:
        key = (item['entity'], item['type'])
        if key in merged_data:
            merged_data[key] = merge_entities(merged_data[key], item)
        else:
            merged_data[key] = item

    # Convert merged data back to list
    merged_data_list = list(merged_data.values())

    return json.dumps(merged_data_list, indent=4)


  def proccessDocument(self, document):
    for paragraphSet in document:
      print(len(paragraphSet))
      self.processInput(paragraphSet, "document")
    self.consolidate_stm_to_ltm()
    print(f"STM Len: {len(self.ShortTermMemory)}.")
    print(f"LTM Len: {len(self.LongTermMemory)}.")


  def processPrompt(self, prompt):
    pass
  '''
  def processInput(self, input, inputType="document"):
    stm_size = 0
    read_pass = 0
    if inputType == "document":
      self.ShortTermMemory.append(self.text_to_stm(variable=input, question=self.Q_KG, api_key=userdata.get('GeminiAPIKey'), model_name="gemini-1.5-flash"))
      for textString in self.ShortTermMemory:
        stm_size += len(textString)
      if stm_size > self.STM_MAX:
        self.consolidate_stm_to_ltm()
    elif inputType == "prompt":
      self.ShortTermMemory.append("TBD")
    else:
      raise ValueError("Invalid input type. Must be 'document' or 'prompt'.")
  '''
  def processInput(self, input, inputType="document"):
    stm_size = 0
    read_pass = 0
    if inputType == "document":
        response = self.text_to_stm(variable=input, question=self.Q_KG, api_key=userdata.get('GeminiAPIKey'), model_name="gemini-1.5-flash")
        # print("AAA:" + response)
        # Attempt to extract JSON and remove triple quotes
        try:
            # Search for JSON enclosed in triple quotes
            json_str = re.search(r'\`\`\`json\s*([\s\S]*?)\s*\`\`\`', response, re.DOTALL).group(1)
            # group(1) captures the content inside the triple quotes
            # print("*******************************************")
            # print(json_str)
            # print("*******************************************")
            json.loads(json_str)  # This line will raise an exception if json_str is not valid JSON
            self.ShortTermMemory.append(json_str)
        except (AttributeError, json.JSONDecodeError):  # Catch both AttributeError and JSONDecodeError
            # Handle cases where the regex doesn't find a match
            print(f"Warning: Could not extract JSON from response: {response}")

        for textString in self.ShortTermMemory:
            stm_size += len(textString)
        if stm_size > self.STM_MAX:
            self.consolidate_stm_to_ltm()
    elif inputType == "prompt":
        self.ShortTermMemory.append("TBD")
    else:
        raise ValueError("Invalid input type. Must be 'document' or 'prompt'.")



In [ ]:
class Document:

    def __init__(self, path, firstParagraphSet=50, remainingParagraphSet=50):
        """
        Initialize the Document object with the given file path.

        Args:
            path (str): The path to a document that will be analyzed. Must be a
                        Word document in docx format.
            firstParagraphSet (int): The number of paragraphs to extract in the first set.
            remainingParagraphSet (int): The number of paragraphs to extract in the subsequent sets.

        Returns:
            None
        """
        self.doc_path = path
        try:
            self.doc = docx.Document(path)
        except Exception as e:
            raise ValueError("Error opening document: " + str(e))
        self.num_paragraphs = len(self.doc.paragraphs)

        self.restart(firstParagraphSet, remainingParagraphSet)
        #self.firstParagraphSet = firstParagraphSet
        #self.remainingParagraphSet = remainingParagraphSet
        #self.iteration = 0
        #self.current_paragraph = 0
        #self.currentParagraphSet = self.firstParagraphSet

    def __iter__(self):
        """
        Make the Document object an iterator.

        Returns:
            self: The iterator object itself.
        """
        return self

    def __next__(self):
        """
        Retrieve the next set of paragraphs from the document.

        Returns:
            str: The requested number of paragraphs as a single string. If there are fewer
                 paragraphs remaining, all available paragraphs are returned. If none are
                 available, StopIteration is raised.
        """
        if self.current_paragraph >= self.num_paragraphs:
            raise StopIteration
        else:
            end_paragraph = min(self.current_paragraph + self.currentParagraphSet, self.num_paragraphs)
            extracted_text = [self.doc.paragraphs[i].text for i in range(self.current_paragraph, end_paragraph)]
            retval = '\n'.join(extracted_text)
            self.current_paragraph = end_paragraph
            if self.iteration == 0:
                self.currentParagraphSet = self.remainingParagraphSet
            self.iteration += 1
            return retval

    def __str__(self):
        """
        Provide a string representation of the Document object.

        Returns:
            str: A string describing the document, including its path and the number of paragraphs.
        """
        return f"Document Path: {self.doc_path}\nTotal Paragraphs: {self.num_paragraphs}\nFirst Paragraph Set Size: {self.firstParagraphSet}\nRemaining Paragraph Set Size: {self.remainingParagraphSet}"

    def restart(self, firstParagraphSet=50, remainingParagraphSet=50):
        """
        Restart the iteration process with new paragraph set sizes.

        Args:
            firstParagraphSet (int): The number of paragraphs to extract in the first set.
            remainingParagraphSet (int): The number of paragraphs to extract in the subsequent sets.

        Returns:
            None
        """
        self.firstParagraphSet = firstParagraphSet
        self.remainingParagraphSet = remainingParagraphSet
        self.iteration = 0
        self.current_paragraph = 0
        self.currentParagraphSet = self.firstParagraphSet

    def get_word_count(self):
        """
        Calculate the total number of words in the document.

        Returns:
            int: The total word count.
        """
        total_words = sum(len(paragraph.text.split()) for paragraph in self.doc.paragraphs)
        return total_words

    def search_keyword(self, keyword):
        """
        Search for a keyword in the document and return the paragraphs containing it.

        Args:
            keyword (str): The keyword to search for.

        Returns:
            list: A list of paragraphs containing the keyword.
        """
        paragraphs_with_keyword = [paragraph.text for paragraph in self.doc.paragraphs if keyword in paragraph.text]
        return paragraphs_with_keyword

    def replace_keyword(self, old_keyword, new_keyword):
        """
        Replace a keyword in the document with a new keyword.

        Args:
            old_keyword (str): The keyword to be replaced.
            new_keyword (str): The new keyword to replace the old one.

        Returns:
            None
        """
        for paragraph in self.doc.paragraphs:
            if old_keyword in paragraph.text:
                paragraph.text = paragraph.text.replace(old_keyword, new_keyword)

    def get_paragraph(self, index):
        """
        Retrieve a specific paragraph by its index.

        Args:
            index (int): The index of the paragraph to retrieve.

        Returns:
            str: The text of the specified paragraph.
        """
        if index < 0 or index >= self.num_paragraphs:
            raise IndexError("Paragraph index out of range.")
        return self.doc.paragraphs[index].text

    def save(self, path=None):
        """
        Save the document to a file.

        Args:
            path (str): The file path to save the document. If None, saves to the original path.

        Returns:
            None
        """
        save_path = path if path else self.doc_path
        self.doc.save(save_path)

    def get_statistics(self):
        """
        Retrieve statistics about the document.

        Returns:
            dict: A dictionary containing various statistics about the document.
        """
        statistics = {
            "Total Paragraphs": self.num_paragraphs,
            "Total Words": self.get_word_count(),
            "First Paragraph Set Size": self.firstParagraphSet,
            "Remaining Paragraph Set Size": self.remainingParagraphSet,
        }
        return statistics


In [ ]:
file_path = "/content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx"
aDocument1 = Document(file_path, 50, 50)
print(aDocument1.get_word_count())
#aList = aDocument1.search_keyword("sewer")
print(aDocument1)
for paragraph in aDocument1:
  print(len(paragraph))
aDocument1.restart(25, 50)
print(aDocument1)
for paragraph in aDocument1:
  print(len(paragraph))

21964
Document Path: /content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx
Total Paragraphs: 1259
First Paragraph Set Size: 50
Remaining Paragraph Set Size: 50
4008
8111
4424
3851
7507
2822
3625
361
5929
6175
6842
9301
8966
9026
7484
7425
4333
4706
7284
5138
9153
6192
2122
2122
1702
8
Document Path: /content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx
Total Paragraphs: 1259
First Paragraph Set Size: 25
Remaining Paragraph Set Size: 50
1141
5914
7969
2842
6821
3861
4254
1570
3421
5270
6566
8815
9339
8827
8558
8347
4661
4472
6725
5624
7010
7446
4237
1119
3263
545


In [ ]:
file_path = "/content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx"
aDocument = Document(file_path, 50, 50)
aMind = Mind()
aMind.proccessDocument(aDocument)
print("Done")

4008
8111
4424
3851
Consolidating STM to LTM
7507
2822
3625
361
5929
6175
Consolidating STM to LTM
6842
9301
8966
Consolidating STM to LTM
9026
7484
7425
Consolidating STM to LTM
4333
4706
7284
5138
9153
Consolidating STM to LTM
6192
2122
2122
1702
8
Consolidating STM to LTM
STM Len: 0.
LTM Len: 6.
Done


In [ ]:

# Example usage
json_str1 = '''
[
    {
        "entity": "Person",
        "type": "Individual",
        "properties": {
            "name": "John",
            "age": 30
        },
        "relationships": {
            "spouse": "Jane"
        }
    },
    {
        "entity": "Company",
        "type": "Organization",
        "properties": {
            "name": "Tech Corp",
            "location": "New York"
        },
        "relationships": {
            "CEO": "John"
        }
    }
]
'''

json_str2 = '''
[
    {
        "entity": "Person",
        "type": "Individual",
        "properties": {
            "occupation": "Engineer",
            "hobbies": ["Reading", "Hiking"]
        },
        "relationships": {
            "children": ["Alice", "Bob"]
        }
    },
    {
        "entity": "Company",
        "type": "Organization",
        "properties": {
            "name": "Innovate Inc",
            "location": "San Francisco"
        },
        "relationships": {
            "CEO": "Alice"
        }
    }
]
'''

merged_json_str = merge_jsons(json_str1, json_str2)
print(merged_json_str)


In [ ]:
file_path = "/content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx"
aDocument = Document(file_path)
try:
  short_term_memory = aDocument.convert_to_knowledge_graph(set_limit=2)
except Exception as e:
  print(f"An error occurred: {e}")
else:
  print(short_term_memory)
finally:
  print("Done")


The input was 16543 which resulted in otput of 8006.
```json
[
  {
    "entity": "Conewago Township Sewer Authority",
    "type": "Organization",
    "properties": {
      "name": "Conewago Township Sewer Authority",
      "abbreviation": "CTSA",
      "establishment_date": "8-22-1989"
    },
    "relationships": [
      {
        "relationship": "adopts",
        "target": "Rules and Regulations"
      }
    ]
  },
  {
    "entity": "Rules and Regulations",
    "type": "Document",
    "properties": {
      "title": "Conewago Township Sewer Authority Rules and Regulations",
      "adoption_date": "9-28-2021"
    },
    "relationships": [
      {
        "relationship": "adopted_by",
        "target": "Conewago Township Sewer Authority"
      }
    ]
  },
  {
    "entity": "Articles of Incorporation",
    "type": "Document",
    "properties": {
      "description": "Specifies the name, purpose, and structure of the Conewago Township Sewer Authority"
    },
    "relationships": [
      {

In [ ]:
file_path = "/content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx"
aDocument = Document(file_path)
try:
  result = aDocument.get_paragraph_count(set_limit=4)
except Exception as e:
  print(f"An error occurred: {e}")
else:
  print(result)
finally:
  print("Done")

The input was 27901 which resulted in otput of 232.
The provided text gives four different paragraph counts: 15, 12, 16, and 16.  Adding these together: 15 + 12 + 16 + 16 = 59

Therefore, assuming the estimates are correct, the total count of paragraphs across all estimations is 59.

Done


In [ ]:
file_path = "/content/drive/MyDrive/Praxis/Conewago Township Sewer Authority, PA.docx"
aDocument = Document(file_path)
try:
  result = aDocument.get_top_words(set_limit=4)
except Exception as e:
  print(f"An error occurred: {e}")
else:
  print(result)
finally:
  print("Done")

Okay, I'm ready to analyze the text and provide the word counts, excluding common function words.

Here's the breakdown of the 10 most frequent words in the provided text, after excluding function words (like "the," "a," "is," "of," "and," "shall," "be," "to," "in," "at," "by," "as," "for"):

Word | Count
------- | --------
Authority | 20
meeting | 9
members | 7
Township | 4
Conewago | 4
meetings | 4
member | 3
regular | 3
purpose | 2
Articles | 2
Okay, I can do that for you. I need to first define which words to exclude as "function words" and then I can tally the occurrence of each word in the provided text.

Here's the breakdown:

**1. Defining "Function Words" (Exclusion List):**

For this analysis, I will exclude the following types of words, as they are generally considered function words:

*   **Articles:** a, an, the
*   **Prepositions:** of, to, in, for, on, at, by, with, from, as, into, upon
*   **Pronouns:** he, she, it, him, her, his, its, they, them, their, i, me, my, mine